# Qiskit Patterns

In this how-to we will learn about Qiskit patterns and quantum approximate optimization. Qiskit Patterns are a workflow to execute a task on a Quantum computer. They comprise four steps

1. Map the classical inputs to a quantum problem
2. Optimize problem for quantum execution
3. Execute using Qiskit Runtime primitives
4. Post-process, return result in classical format

In Step 1. we will take a combinatorial optimization problem and formulate it in terms of finding the ground state of an Ising Hamiltonian. This reformulated prblem can be understood by a quantum computer. In Step 2. we will prepare the quantum circuits to execute on the quantum computer.
In Step 3. we will call the `Sampler` primitive in Qiskit to draw samples from the quantum circuits that Step 2. prepared. Finally, under Step 4. we convert the samples from Step 3. into the solution of our combinatorial optimization problem. 

## 1. Map the classical inputs to a quantum problem

We are interested in solving a classical combinatorial optimization problem which has the form

\begin{align}
\min_{x\in \{0, 1\}^n}f(x)
\end{align}

Here, the vector $x$ are the $n$ decision variables. As you can see, there is nothing relating to quantum computing here yet. We therefore need to reformulate this problem into something that a quantum computer can understand. To be more concrete, we will consider a Quadratic Unconstrained Binary Optimization problem with the form

\begin{align}
\min_{x\in \{0, 1\}^n}x^T Q x,
\end{align}
where $Q$ is a $n\times n$ matrix of real numbers. To start, we will convert the binary variables $x_i$ to variables $z_i\in\{-1, 1\}$ by doing
\begin{align}
x_i = \frac{1-z_i}{2}.
\end{align}
Here, for example, we see that if $x_i$ is $0$ then $z_i$ is $1$. When we substitute the $x_i$'s for the $z_i$'s in the QUBO above, we obtain the equivalent formulations for our optimization task
\begin{align}
\min_{x\in\{0,1\}^n} x^TQx\Longleftrightarrow \min_{z\in\{-1,1\}^n}z^TQz + b^Tz
\end{align}
The details of the computation are shown in Appendix A below. Here, $b$ depends on $Q$. Note that to obtain $z^TQz + b^Tz$ we dropped an irrelevant factor of 1/4 and a constant offset of $n^2$ which do not play a role in the optimization. Now, to obtain a quantum formulation of the problem we promot the $z_i$ variables to a Pauli $Z$ matrix, i.e., a $2\times 2$ matrix of the form
\begin{align}
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
\end{align}
When we substitute these matrices in the QUBO above we obtain the following Hamiltonian
\begin{align}
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
\end{align}
We refer to this Hamiltonian as the cost function Hamiltonian. It has the property that its gound state corresponds to the solution that minimizes the cost function $f(x)$.
Therefore, to solve out optimization problem we now need to prepare the ground state of $H_C$ (or a state with a high overlap with it) on the quantum computer. Then, sampling from this state will, with a high probability, yield the solution to $min f(x)$.

**TODO** Load a file with the LP and convert to an Ising Hamiltonian to feed into step 2.

## 2. Optimize problem for quantum execution

**TODO** Take the Ising Hamiltonian from Step 1. and make circuits.

## 3. Execute using Qiskit Runtime primitives

**TODO** Call the sampler on the circuits form Step 2.

## 4. Post-process, return result in classical format

**TODO** Plot the distribution of results.

## Discussion and conclusion

## Appendix A: Reformulation in spin variables

Here, we rewrite the QUBO $x^TQx$ in terms of spin-variables $x_i=(1-z_i)/2$.
\begin{align}
x^TQx=\sum_{ij}Q_{ij}x_ix_j=\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j)=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
\end{align}
If we write $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ and remove the prefactor and the constant $n^2$ term we arrive at the two equivalent formulations of the same optimization problem
\begin{align}
\max_{x\in\{0,1\}^n} x^TQx\Longleftrightarrow \max_{z\in\{-1,1\}^n}z^TQz + b^Tz
\end{align}

## Appendix B: Quantum notation

The $Z$ matrices are imbedded in the quantum computer's computational space, i.e., a Hilbert space of size $2^n\times 2^n$. Therefore, you should understand terms such as $Z_iZ_j$ as the tensor product $Z_i\otimes Z_j$ imbedded in the $2^n\times 2^n$ Hilbert space. For example, in a problem with five decision variables the term $Z_1Z_3$ is understood to mean $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ where $I$ is the $2\times 2$ identity matrix.